In [1]:
import sys
sys.path.append('/home/lumargot/ShapeAXI/src')
from shapeaxi.saxi_dataset import SaxiDataModuleVF

/home/lumargot/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
import torch
from torch import nn
from pytorch3d.ops import sample_points_from_meshes
import numpy as np

import plotly.graph_objects as go
import plotly.express as px
from pytorch3d.vis.plotly_vis import plot_scene
from pytorch3d.structures import (
    Meshes,
    Pointclouds,)
    
import sys
sys.path.append('/home/lumargot/ShapeAXI/src')
from shapeaxi.saxi_dataset import SaxiDataModuleVF, SaxiPCDataModule
import pandas as pd
from tqdm import tqdm 

In [3]:
from shapeaxi.saxi_transforms import TrainTransform, EvalTransform
from shapeaxi import utils 

In [4]:
df = pd.read_csv("/CMF/data/lumargot/DCBIA/Combined/csv/condyles_4classes_consensus_Marcela.csv")

In [5]:
surf_scales = []
pbar = tqdm(df.iterrows(), total=len(df))

for idx, row in pbar:
    surf = utils.ReadSurf(row['surf'])

    unit_surf, surf_mean, surf_scale = utils.ScaleSurf(surf)
    surf_scales.append(surf_scale)

    pbar.set_description('surf_scale={surf_scale}'.format(surf_scale=surf_scale))

surf_scales = np.array(surf_scales)
min_scale = np.min(surf_scales)


surf_scale=0.0609600900973719: 100%|██████████| 389/389 [00:01<00:00, 361.84it/s]  


In [6]:
min_scale

0.05071661913144526

In [7]:
batch_size = 1
dm = SaxiDataModuleVF(
    csv_train="/CMF/data/lumargot/DCBIA/Combined/csv/condyles_4classes_consensus_Marcela.csv",
    csv_valid="/CMF/data/lumargot/DCBIA/Combined/csv/condyles_4classes_consensus_Marcela.csv",
    csv_test="/CMF/data/lumargot/DCBIA/Combined/csv/condyles_4classes_consensus_Marcela.csv",
    mount_point="",
    batch_size=batch_size,
    num_workers=1,
    surf_column="surf",
    drop_last=False,
    train_transform = EvalTransform(scale_factor=min_scale),
    valid_transform = EvalTransform(scale_factor=min_scale),
    test_transform = EvalTransform(scale_factor=min_scale),
    )
dm.setup()
train_loader = dm.train_dataloader()

In [8]:
from tqdm import tqdm
from shapeaxi.saxi_nets import HilbertSort3D

In [9]:
data_out = '/CMF/data/lumargot/DCBIA/Combined/4096_npy/'

num_points = 64
N = num_points*num_points
# N=4096

In [10]:
list_npy = []
for idx, batch in tqdm(enumerate(train_loader), total=len(train_loader)):
  V,F = batch

  mesh_batch = Meshes(verts=V, faces=F)
  P = sample_points_from_meshes(mesh_batch, N)

  # sorter_pt = HilbertSort3D(radius=1.25, bins=num_points)
  # p_sorted_pt = sorter_pt(P)[0]
  # p_rgb = p_sorted_pt.view(-1, num_points, num_points, 3).squeeze(0).detach().cpu().numpy()
  # p_rgb = p_rgb - p_rgb.min()
  # p_rgb = p_rgb / p_rgb.max()

  

  filepath = train_loader.dataset.getSurfPath(idx)
  sample_name = os.path.splitext(os.path.basename(filepath))[0]
  outfile = os.path.join(data_out, sample_name +'.npy')

  np.save(outfile, P)
  # list_npy.append(outfile)

# df['hilbert_space'] = list_npy

100%|██████████| 389/389 [00:12<00:00, 32.11it/s]


In [11]:
df.to_csv(input_csv)

NameError: name 'input_csv' is not defined

In [ ]:
dir_csv = "/CMF/data/lumargot/DCBIA/Combined/csv/"

In [ ]:
for csv_name in os.listdir(dir_csv):
  if 'Marcela_' in csv_name:
    file_i =  os.path.join(dir_csv, csv_name)
    df_i = pd.read_csv(file_i)

    df_i['hilbert_space'] = df_i.apply(lambda x: os.path.join(data_out, os.path.splitext(os.path.basename(x['surf']))[0] +'.npy'), 
                                       axis=1)
    # for idx, row in df_i.iterrows():
      # print(os.path.exists(row['hilbert_space']), row['hilbert_space'])

    df_i.to_csv(file_i)

In [ ]:
df_i

,Unnamed: 0.1,Unnamed: 0,id,class_consensus,surf,bin_class,hilbert_space
0,76,76,DJ110124R,1,/work/jprieto/data/DCBIA/4_Clipped_VTKs_323_co...,1,/CMF/data/lumargot/DCBIA/Combined/npy/DJ110124...
1,139,139,MM120610L,3,/work/jprieto/data/DCBIA/4_Clipped_VTKs_323_co...,1,/CMF/data/lumargot/DCBIA/Combined/npy/MM120610...
2,107,107,HH100615L,3,/work/jprieto/data/DCBIA/4_Clipped_VTKs_323_co...,1,/CMF/data/lumargot/DCBIA/Combined/npy/HH100615...
3,298,298,p0020 LeftMI,1,/CMF/data/lumargot/DCBIA/DJD/data/p0020_T1_Mod...,1,/CMF/data/lumargot/DCBIA/Combined/npy/p0020_T1...
4,274,274,r_189_T1 Right,0,/CMF/data/lumargot/DCBIA/DJD/data/r_189_T1_Rig...,0,/CMF/data/lumargot/DCBIA/Combined/npy/r_189_T1...
...,...,...,...,...,...,...,...
72,216,216,r_6_T1 Right,1,/CMF/data/lumargot/DCBIA/DJD/data/r_6_T1_Right...,1,/CMF/data/lumargot/DCBIA/Combined/npy/r_6_T1_R...
73,330,330,r_94_T1 LeftMI,0,/CMF/data/lumargot/DCBIA/DJD/data/r_94_T1_Left...,0,/CMF/data/lumargot/DCBIA/Combined/npy/r_94_T1_...
74,66,66,CMLM,2,/work/jprieto/data/DCBIA/4_Clipped_VTKs_323_co...,1,/CMF/data/lumargot/DCBIA/Combined/npy/CMLM.npy
75,90,90,FK120507L,3,/work/jprieto/data/DCBIA/4_Clipped_VTKs_323_co...,1,/CMF/data/lumargot/DCBIA/Combined/npy/FK120507...


In [ ]:
row = df_i.iloc[0]

In [ ]:
row['surf']

'/work/jprieto/data/DCBIA/4_Clipped_VTKs_323_condyles/DJ110124R.vtk'

In [ ]:
df_i['hilbert_space'] = df_i.apply(lambda x: os.path.join(data_out, os.path.splitext(os.path.basename(x['surf']))[0] +'.npy'), axis=1)

In [ ]:
file_test = os.path.join(data_out, os.path.splitext(os.path.basename(row['surf']))[0] +'.npy')

In [ ]:
os.path.exists(file_test)

True